# Part I. ETL Pipeline for Pre-Processing the Files

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create & Set Keyspace

In [ ]:
CREATE_KEYSPACE = """
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }"""

# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.execute(CREATE_KEYSPACE)

except Exception as e:
    print(e)
    
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [ ]:
# CREATE TABLES
create_song_session = '''
CREATE TABLE IF NOT EXISTS sessions(session_id INT,
                                    item_in_session INT,
                                    artist_name TEXT,
                                    song_title TEXT,
                                    song_duration DECIMAL,
                                    PRIMARY KEY(session_id,
                                                item_in_session))'''
create_user_listens = '''
CREATE TABLE IF NOT EXISTS user_listens(user_id INT,
                                        session_id INT,
                                        item_in_session INT,
                                        artist_name TEXT,
                                        song_title TEXT,
                                        user_first_name TEXT,
                                        user_last_name TEXT,
                                        PRIMARY KEY(user_id,
                                                    session_id,
                                                    item_in_session))'''
create_song_listens = '''
CREATE TABLE IF NOT EXISTS song_listens(song_title TEXT,
                                        user_id INT,
                                        user_first_name TEXT,
                                        user_last_name TEXT,
                                        PRIMARY KEY(song_title,
                                                    user_id))'''
try:
    session.execute(create_song_session)
    session.execute(create_user_listens)
    session.execute(create_song_listens)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

#query = "INSERT INTO music_library (year, artist_name, album_name)"
#query = query + " VALUES (%s, %s, %s)"
        query1 = "INSERT INTO sessions(session_id, item_in_session, artist_name, song_title, song_duration)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        
        query2 = "INSERT INTO user_listens(user_id, session_id, item_in_session, artist_name, song_title, user_first_name, user_last_name)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        query3 = "INSERT INTO song_listens(song_title, user_id, user_first_name, user_last_name)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"

        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
#question1 query
question1 = """
SELECT artist_name, song_title, song_duration
FROM sessions 
WHERE session_id=338 AND item_in_session=4
"""

#question2 query
question2 = """
SELECT artist_name, song_title, user_first_name, user_last_name
FROM user_listens 
WHERE user_id=10 AND session_id=182
"""

#question3 query
question3 = """
SELECT user_first_name, user_last_name
FROM song_listens 
WHERE song_title = 'All Hands Against His Own'
"""

try:
    rows1 = session.execute(question1)
    rows2 = session.execute(question2)
    rows3 = session.execute(question3)
except Exception as e:
    print(e)

print("Question 1 Answer: ")
print("| artist_name | song_title | song_title |")
print("____________________________________________________________")
for row in rows1:
    print('|', row.artist_name, '|', row.song_title, '|', row.song_duration, '|' )
print("\n")
print("Question 2 Answer: ")
print("| artist_name | song_title | user_first_name | user_last_name |")
print("____________________________________________________________________________")
for row in rows2:
    print('|', row.artist_name, '|', row.song_title, '|', row.user_first_name, '|', row.user_last_name, '|') 
print("\n")
print("Question 3 Answer: ")
print("| user_first_name | user_last_name |")
print("__________________________________")
for row in rows3:
    print('|', row.user_first_name, '|', row.user_last_name, '|') 

### Drop the tables before closing out the sessions

In [ ]:
query1 = "drop table sessions"
query2 = "drop table user_listens"
query3 = "drop table song_listens"

DROP_TABLE = [query1, query2, query3]
try:
    for query in DROP_TABLE:
        session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()